<a href="https://colab.research.google.com/github/johnstaib2/NodeMCU_matrix_text/blob/master/problem_backprop_F23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 – Backprop

In [ ]:
#@title Library Imports [do not change]

import importlib
!git clone https://www.github.com/rycolab/intro-nlp-f23.git
utils = importlib.import_module("intro-nlp-f23.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

fatal: destination path 'intro-nlp-f23' already exists and is not an empty directory.


In [ ]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "0" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': '(x^2 - 1) * (y+3)', 'in_vars': {'x': 4.0, 'y': 2.0}, 'output': 75.0, 'derivative': {'x': 40.0, 'y': 15.0}}
[[['x', '^', 2], '-', 1], '*', ['y', '+', 3]] {'y': 2.0, 'x': 4.0}


In [ ]:
from IPython.core.interactiveshell import available_events
#@title ToDo1: Building

class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)
        self.infix = infix
        self.graph = Graph()

        ## ToDO: implement and set self.graph
        #first we need the nodes for the in_variables
        for i in in_vars.keys():
          self.graph.addKey(i)
          ver = Node(i)
          self.graph.addVertex(ver)
        output_node=self.recursiveBuilder(infix,in_vars)
        self.graph.output_node=next((node for node in self.graph.nodes if node.id==output_node),None)


    def recursiveBuilder(self,infix,invars):
        # first give this entire operation a variable
        name=self.avail_vars.pop()
        vertex = Node(name)
        self.graph.addVertex(vertex)
        self.graph.addKey(name)

        if len(infix)==1: #literal, could be like f(x)=x -> ['x']
           if (infix[0] in self.graph.keys):
            node = next((el for el in self.graph.nodes if el.id == infix[0]), None)
            node.addNeighbor(name)
            vertex.arg1=node.id
           else: # it is a constant
            vertex.value=infix[0]


        if len(infix)==2: # then it is a function on only one variable, namely the second operation
           vertex.operation=infix[0]
           #the second node:
           if isinstance(infix[1], list):
              arg1 = self.recursiveBuilder(infix[1],invars)
              node = next((el for el in self.graph.nodes if el.id == arg1), None)
              node.addNeighbor(name)
              vertex.arg1=arg1

           else:
            # it is a literal, check if initial variable or constant
            if (infix[1] in self.graph.keys):
              node = next((el for el in self.graph.nodes if el.id == infix[1]), None)
              node.addNeighbor(name)
              vertex.arg1=node.id
            else: #its a constant
              vertex.arg1_val=infix[1]



        # more complicated, might need recursion again.
        if len(infix)==3:
          # process first argument
          if isinstance(infix[0], list):
              arg1 = self.recursiveBuilder(infix[0],invars)
              node = next((el for el in self.graph.nodes if el.id == arg1), None)
              node.addNeighbor(name)
              vertex.arg1=arg1
          else:
            # if it is an input variable, make an edge from that variable to this vertex
            if (infix[0] in self.graph.keys):
              node = next((el for el in self.graph.nodes if el.id == infix[0]), None)
              node.addNeighbor(name)
              vertex.arg1=infix[0]
            else:
              #if constant, set second first argument of the vertex to the value
              vertex.arg1_val=infix[0]

          # store the operation
          vertex.operation=infix[1]

          #now process the second term
          if isinstance(infix[2], list):
              arg2 = self.recursiveBuilder(infix[2],invars)
              node = next((el for el in self.graph.nodes if el.id == arg2), None)
              node.addNeighbor(name)
              vertex.arg2 = arg2
          else:
            # if it is an input variable, make an edge from that variable to this vertex
            if (infix[2] in self.graph.keys):
              node = next((el for el in self.graph.nodes if el.id == infix[2]), None)
              node.addNeighbor(name)
              vertex.arg2 = infix[2]
            else:
              #if constant, set second second argument of the vertex to the value
              vertex.arg2_val=infix[2]

        return name



class Node():
    def __init__(self,id):
        self.id =id
        self.neighbors = []
        self.ancestors=[]
        self.value=None
        self.operation = None
        self.arg1 = None
        self.arg2 = None
        self.arg1_val = None
        self.arg2_val = None


    def __lt__(self, other):
        # Define the comparison based on the 'value' attribute
       return self.id < other.id

    def addNeighbor(self,id):
        if self.neighbors == []:
           self.neighbors=[id]
        else:
          self.neighbors.append(id)

    def addAncestor(self,id):
        if self.ancestors == []:
           self.ancestors=[id]
        else:
          self.ancestords.append(id)


    def setArg(self,value,name):
        if self.arg1==name:
           self.arg1_val=value
        else:
           if self.arg2 == name:
              self.arg2_val=value
           else:
            print(name," should not point here (",self.id,")")


class Graph():
    def __init__(self) -> None:
        self.n=0
        self.nodes=set()
        self.keys=set()
        self.output_node=None

    def addVertex(self,v):
        self.nodes.add(v)
        self.n=self.n+1
    def addKey(self,key):
        self.keys.add(key)





if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.graph)

In [ ]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        return math.log(a)

    def df(self, a, b = None):
        return 1.0/a

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        return a/b

    def df(self, a, b):
        return [1/b, -a/(b ** 2)]

class Add(Operator):

    def f(self, a, b):
        return a+b

    def df(self, a, b = None):
        return 1

class Sub(Operator):

    def f(self, a, b = None):
        return a-b

    def df(self, a, b = None):
        return -1

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        return math.sin(a)

    def df(self, a, b=None):
        return math.cos(a)

class Cos(Operator):

    def f(self, a, b=None):
        return math.cos(a)

    def df(self, a, b=None):
        return -math.sin(a)

In [ ]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self, ):
        in_vars_nodes = {el for el in self.graph.nodes if el.id in in_vars}

        for el in self.in_vars:
          node = next((no for no in in_vars_nodes if no.id == el), None)
          value=self.in_vars[el]
          node.value=value
          for neighbor in node.neighbors:
             neighbor_node=next((no for no in self.graph.nodes if no.id == neighbor), None)
             neighbor_node.setArg(value,el)

        #now that we updated the in variables, work on the rest in back to front order
        nodes_without_in_vars = self.graph.keys-in_vars.keys()
        sorted_nodes = sorted(nodes_without_in_vars, reverse=True)


        for i in range(len(sorted_nodes)):
          node = next((el for el in self.graph.nodes if el.id ==sorted_nodes[i]),None)
          fn = self.fn_map[node.operation]
          node.value = fn.f(node.arg1_val,node.arg2_val)

          for neighbor in node.neighbors:
             neighbor_node=next((no for no in self.graph.nodes if no.id == neighbor), None)
             neighbor_node.setArg(node.value,node.id)

        self.output= self.graph.output_node.value


    ## backward execution____________________________

    def backward(self, ):
        nodes_without_in_vars = self.graph.keys-in_vars.keys()
        sorted_nodes = sorted(nodes_without_in_vars, reverse=False)
        for i in range(len(sorted_nodes)-1):
          node = sorted_nodes[i]

        self.derivative = {}


if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

75.0
{}


In [ ]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)


0: problem: (x^2 - 1) * (y+3), in_vars: {'x': 4.0, 'y': 2.0}
SUCCESS output: 75.0
FAILURE derivative: {} != {'x': 40.0, 'y': 15.0}


In [ ]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 2.0, 'y': 4.0}
SUCCESS output: 0.5
FAILURE derivative: {} != {'x': 0.25, 'y': -0.125}

1: problem: exp(x) - (y * 4), in_vars: {'x': 3.0, 'y': 3.0}
SUCCESS output: 8.09
FAILURE derivative: {} != {'x': 20.09, 'y': -4.0}

2: problem: (x^2 - 1) * (y+3), in_vars: {'x': 4.0, 'y': 2.0}
SUCCESS output: 75.0
FAILURE derivative: {} != {'x': 40.0, 'y': 15.0}


AttributeError: ignored